# 0. Load Dependencies

In [ ]:
### Import the vulnerability toolkit library
import sys
repoDir = 'C:/Users/Moayad/Documents/GitHub/stickModel'
sys.path.insert(1, f'{repoDir}')
from stickModel import stickModel
from utils import *
from units import *
from postprocessors import *
from plotters import *
from im_calculator import *

### Import other libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import os
from math import sqrt, pi

# 1. User Input

In [ ]:
### Define Capacity Curves Directory
capCurvesDir= f'{repoDir}/raw/ip-capacities'

### Define Building Class
currentBuildingClass='CR_LFINF+CDM+DUM_H4'

### Define Ground-Motion Records Directory
gmrsDir = f'{repoDir}/examples/records'

### Define Output Directory
outDir = f'{repoDir}/examples/'

# 2. Define the SDOF-to-MDOF Calibration Function

In [ ]:
def calibrateModel(nst,gamma,sdofCapArray):
        
    # Initialise the mode shape
    phi_mdof = np.linspace(1/nst, 1, nst)
    print('Assumed Mode-Shape:',phi_mdof)
    
    # Get the masses
    mass = 1/sum(phi_mdof)
    print('Floor Mass:',mass)
    
    # Assign the MDOF mass
    flm_mdof = [mass]*nst

    ### Get the MDOF Capacity Curves Storey-Deformation Relationship
    rows, columns = np.shape(sdofCapArray)
    stD_mdof = np.zeros([nst,rows])
    stF_mdof = np.zeros([nst,rows])
    for i in range(nst):
        # get the displacement or spectral displacement arrays at each storey
        stF_mdof[i,:] = sdofCapArray[:,1].transpose()*gamma*units.g*sum(flm_mdof)
        # get the force or spectral acceleration arrays at each storey
        stD_mdof[i,:] = sdofCapArray[:,0].transpose()*gamma*phi_mdof[i]
        
    ### Get the elastic stiffness of each floor
    elasticStiffness = []
    for i in range(nst):
        elasticStiffness.append(stF_mdof[i,0]/stD_mdof[i,0])
        print('Elastic Stiffness of',f'{i+1}-th Storey:', elasticStiffness[i])
        
    ### Estimate the elastic period
    elasticPeriod = 2*pi*np.sqrt(sum(flm_mdof)/sum(elasticStiffness))

    ### Prepare the SDOF model
    flh_sdof = [2.8]*1 # arbitray height 
    flm_sdof = [1.0]*1 # unit mass (1 tonne)
    rows, columns = np.shape(sdofCapArray)
    stD_sdof = np.zeros([1,rows])
    stF_sdof = np.zeros([1,rows])
    for i in range(1):
        # get the displacement or spectral displacement arrays at each storey
        stF_sdof[i,:] = sdofCapArray[:,1].transpose()*gamma*units.g
        # get the force or spectral acceleration arrays at each storey
        stD_sdof[i,:] = sdofCapArray[:,0].transpose()*gamma
    
    ### Compile the SDOF model
    modelSDOF = stickModel(1,flh_sdof,flm_sdof,stD_sdof,stF_sdof)    # Build the model
    modelSDOF.mdof_initialise()                                      # Initialise the domain
    modelSDOF.mdof_nodes()                                           # Construct the nodes
    modelSDOF.mdof_fixity()                                          # Set the boundary conditions 
    modelSDOF.mdof_loads()                                           # Assign the loads
    modelSDOF.mdof_material()                                        # Assign the nonlinear storey material
    modelSDOF.plot_model()                                           # Visualise the model                               
    modelSDOF.do_gravity_analysis()                                  # Do gravity analysis
    sdofT = modelSDOF.do_modal_analysis(num_modes = 1)               # Do modal analysis and get period of vibration

    ### Run static pushover analyses on SDOF model
    ref_disp = 0.01
    disp_scale_factor = 100
    push_dir = 1
    spoSDOF_disps, spoSDOF_rxn= modelSDOF.do_spo_analysis(ref_disp,disp_scale_factor,push_dir)

    ### Compile the MDOF
    flh_mdof = [2.8]*nst
    modelMDOF = stickModel(nst,flh_mdof,flm_mdof,stD_mdof,stF_mdof)  # Build the model
    modelMDOF.mdof_initialise()                                      # Initialise the domain
    modelMDOF.mdof_nodes()                                           # Construct the nodes
    modelMDOF.mdof_fixity()                                          # Set the boundary conditions 
    modelMDOF.mdof_loads()                                           # Assign the loads
    modelMDOF.mdof_material()                                        # Assign the nonlinear storey material
    modelMDOF.plot_model()                                           # Visualise the model                               
    modelMDOF.do_gravity_analysis()                                  # Do gravity analysis
    mdofT = modelMDOF.do_modal_analysis(num_modes = 1)               # Do modal analysis and get period of vibration

    ### Run Static Pushover Analyses on MDOF 
    ref_disp = 0.01
    disp_scale_factor = 20
    push_dir = 1
    spoMDOF_disps, spoMDOF_rxn= modelSDOF.do_spo_analysis(ref_disp,disp_scale_factor,push_dir)
    
    ### Test 1: Check the periods
    print('SDOF Elastic Period from Modal Analysis:', sdofT[0], 's')
    print('MDOF Elastic Period from Modal Analysis:', mdofT[0], 's')
    error1 = (sdofT[0]-mdofT[0])/mdofT[0]
    if error1 < 1e-1:
        print('Period check satisfied!')
        print('Error 1:', error1*100)
    else:
        print('Error 1:', error1*100)
        raise ValueError('MDOF and SDOF periods do not match!! Revise your input')
    
    ### Test 2: Equivalent Masses
    error2 = (np.dot(flm_mdof,phi_mdof)-1.00)/1.00
    if error2 < 1e-1:
        print('Equivalent mass equation verified')
        print('Error 2:', error2*100)
    else:
        print('Error 2:', error2*100)
        #raise ValueError('Equivalent mass equation not satisfied!! Revise your input')
        
    ### Test 3: Transformation Equation
    error3 = (np.dot(flm_mdof,phi_mdof**2)-1/gamma)/(1/gamma)
    if error3 < 1e-1:
        print('Transformation factor equation verified')
        print('Error 3:', error3*100)
    else:
        print('Error 3:', error3*100)
        #raise ValueError('Transformation factor equation not satisfied!! Revise your input')
        
    ### Plot the comparison
    fig=plt.figure()    
    # Plot the individual storeys
    for i in range(nst): 
        storeyD = np.concatenate(([0.0],stD_mdof[i,:]))
        storeyF = np.concatenate(([0.0],stF_mdof[i,:]))
        plt.plot(storeyD,storeyF,'--',label = f'storey {i}')
    
    # Plot the backbone of the SDOF system
    plt.plot(spoSDOF_disps[:,-1],spoSDOF_rxn,'r--',label='Model - SDOF')
    # Plot the calibrated backbone of the MDOF 
    plt.plot(spoMDOF_disps[:,-1],spoMDOF_rxn,label='Model - MDOF')
    plt.grid(visible=True, which='major')
    plt.grid(visible=True, which='minor')
    plt.xlabel('Spectral Displacement, Sd [m]')
    plt.ylabel('Spectral Acceleration, Sa [g]')
    plt.title('Comparison of Capacity Curves')
    plt.legend()
        
    return mdofT, flm_mdof, stD_mdof, stF_mdof


# 3. Prepare the MDOF Model and Run NLTHA

In [ ]:
### Load the building class info
classInfo = pd.read_csv(f'{capCurvesDir}/in_plane_capacity_parameters_table.csv')

### Import the building class info
nst          = int(currentBuildingClass.split('H')[-1])
storeyHeight = classInfo['Storey_height'].loc[classInfo['Building_class']==currentBuildingClass].item()
gamma        = classInfo['Participation_factor'].loc[classInfo['Building_class']==currentBuildingClass].item()

### Import the equivalent SDOF capacity array 
sdofCapArray = np.array(pd.read_csv(f'{capCurvesDir}/{currentBuildingClass}.csv', header = None))[1:,:]

### Calibrate the model and get the period, masses, and strength-deformation relationships of all storeys
T, flm_mdof, stD_mdof, stF_mdof = calibrateModel(nst,gamma, sdofCapArray)

### Get the number of stories and different storey heights
nst = int(currentBuildingClass.split('H')[-1])
flh_mdof = [2.8]*nst  # Storey heights

### Fetch ground motion records
gmrs = sorted_alphanumeric(os.listdir(f'{gmDir}/gmrs'))   # Sort the ground-motion records alphanumerically

### Initialise storage lists
mdof_coll_index_list = []               # List for collapse index
mdof_peak_disp_list  = []               # List for peak floor displacement (returns all peak values along the building height)
mdof_peak_drift_list = []               # List for peak storey drift (returns all peak values along the building height)
mdof_peak_accel_list = []               # List for peak floor acceleration (returns all peak values along the building height)
mdof_max_peak_drift_list = []           # List for maximum peak storey drift (returns the maximum value) 
mdof_max_peak_drift_dir_list = []       # List for maximum peak storey drift directions
mdof_max_peak_drift_loc_list = []       # List for maximum peak storey drift locations
mdof_max_peak_accel_list = []           # List for maximum peak floor acceleration (returns the maximum value)
mdof_max_peak_accel_dir_list = []       # List for maximum peak floor acceleration directions 
mdof_max_peak_accel_loc_list = []       # List for maximum peak floor acceleration locations 
mdof_pga = []; mdof_sa = []             # List for intensity measures

for i in range(len(gmrs)):
    
    ### Compile the SDOF model
    model = stickModel(nst,flh_mdof,flm_mdof,stD_mdof,stF_mdof)            # Build the model
    model.mdof_initialise()                                      # Initialise the domain
    model.mdof_nodes()                                           # Construct the nodes
    model.mdof_fixity()                                          # Set the boundary conditions 
    model.mdof_loads()                                           # Assign the loads
    model.mdof_material()                                        # Assign the nonlinear storey material
    if i==0:
        model.plot_model()                                       # Visualise the model
    else: 
        pass
    model.do_gravity_analysis()                                  # Do gravity analysis
    T = model.do_modal_analysis(num_modes = 1)                   # Do modal analysis and get period of vibration

    ### Define ground motion objects
    fnames = [f'{gmDir}/gmrs/{gmrs[i]}']                                # Ground-motion record names
    fdts = f'{gmDir}/dts/dts_{i}.csv'                                   # Ground-motion time-step names 
    dt_gm = pd.read_csv(fdts)[pd.read_csv(fdts).columns[0]].loc[0]      # Ground-motion time-step
    t_max = pd.read_csv(fdts)[pd.read_csv(fdts).columns[0]].iloc[-1]    # Ground-motion duration

    ### Get intensity measures
    im = intensityMeasure(pd.read_csv(fnames[0]).to_numpy().flatten(),dt_gm) # Initialise the intensityMeasure object
    pga.append(im.get_sa(0.0))                                               # Append PGA values 
    sa.append(im.get_sa(0.3))                                                # Append Sa(0.3s) values
    
    ### Define analysis params and do NLTHA
    dt_ansys = dt_gm                                                         # Set the analysis time-step
    sf = units.g                                                             # Set the scaling factor (if records are in g, a scaling factor of 9.81 m/s2 must be used to be consistent with opensees) 
    collLimit = 10.00                                                        # Set a large number for numerical collapse
    control_nodes, coll_index, peak_drift, peak_accel, max_peak_drift, max_peak_drift_dir, max_peak_drift_loc, max_peak_accel, max_peak_accel_dir, max_peak_accel_loc, peak_disp = model.do_nrha_analysis(fnames, dt_gm, sf, t_max, dt_ansys, collLimit, outDir)

    #######################            
    ### Store the analysis 
    #######################
    mdof_coll_index_list.append(coll_index)
    mdof_peak_drift_list.append(peak_drift)
    mdof_peak_accel_list.append(peak_accel)
    mdof_peak_disp_list.append(peak_disp)
    mdof_max_peak_drift_list.append(max_peak_drift)
    mdof_max_peak_drift_dir_list.append(max_peak_drift_dir)
    mdof_max_peak_drift_loc_list.append(max_peak_drift_loc)
    mdof_max_peak_accel_list.append(max_peak_accel)
    mdof_max_peak_accel_dir_list.append(max_peak_accel_dir)
    mdof_max_peak_accel_loc_list.append(max_peak_accel_loc)
    #######################

    print('================================================================')
    print('============== ANALYSIS COMPLETED: {:d} out {:d} =================='.format(i+1, len(gmrs)))
    print('================================================================')

# 4. Prepare the SDOF Model and Run NLTHA

In [ ]:
### Prepare the SDOF model
nst = 1                   # Single-degree-of-freedom (nst = 1) 
flh = [storeyHeight]*nst  # Storey height
flm = [1.0]*nst           # Storey mass (unit tonne mass for sdof systems)

### Get the storey-strength deformation relationship
rows, columns = np.shape(sdofCapArray)
stD = np.zeros([nst,rows])
stF = np.zeros([nst,rows])
for i in range(nst):
    # get the displacement or spectral displacement arrays at each storey
    stF[i,:] = sdofCapArray[:,1].transpose()*gamma*units.g
    # get the force or spectral acceleration arrays at each storey
    stD[i,:] = sdofCapArray[:,0].transpose()*gamma   

### Fetch ground motion records
gmrs = sorted_alphanumeric(os.listdir(f'{gmDir}/gmrs'))   # Sort the ground-motion records alphanumerically

### Initialise storage lists
sdof_coll_index_list = []               # List for collapse index
sdof_peak_disp_list  = []               # List for peak floor displacement (returns all peak values along the building height)
sdof_peak_drift_list = []               # List for peak storey drift (returns all peak values along the building height)
sdof_peak_accel_list = []               # List for peak floor acceleration (returns all peak values along the building height)
sdof_max_peak_drift_list = []           # List for maximum peak storey drift (returns the maximum value) 
sdof_max_peak_drift_dir_list = []       # List for maximum peak storey drift directions
sdof_max_peak_drift_loc_list = []       # List for maximum peak storey drift locations
sdof_max_peak_accel_list = []           # List for maximum peak floor acceleration (returns the maximum value)
sdof_max_peak_accel_dir_list = []       # List for maximum peak floor acceleration directions 
sdof_max_peak_accel_loc_list = []       # List for maximum peak floor acceleration locations 
sdof_pga = []; sdof_sa = []             # List for intensity measures

for i in range(len(gmrs)):
    
    ### Compile the SDOF model
    model = stickModel(nst,flh,flm,stD,stF)                      # Build the model
    model.mdof_initialise()                                      # Initialise the domain
    model.mdof_nodes()                                           # Construct the nodes
    model.mdof_fixity()                                          # Set the boundary conditions 
    model.mdof_loads()                                           # Assign the loads
    model.mdof_material()                                        # Assign the nonlinear storey material
    if i==0:
        model.plot_model()                                       # Visualise the model
    else: 
        pass
    model.do_gravity_analysis()                                  # Do gravity analysis
    T = model.do_modal_analysis(num_modes = 1)                   # Do modal analysis and get period of vibration
    
    ### Define ground motion objects
    fnames = [f'{gmDir}/gmrs/{gmrs[i]}']                                # Ground-motion record names
    fdts = f'{gmDir}/dts/dts_{i}.csv'                                   # Ground-motion time-step names 
    dt_gm = pd.read_csv(fdts)[pd.read_csv(fdts).columns[0]].loc[0]      # Ground-motion time-step
    t_max = pd.read_csv(fdts)[pd.read_csv(fdts).columns[0]].iloc[-1]    # Ground-motion duration
    
    ### Define analysis params and do NLTHA
    dt_ansys = dt_gm                                                         # Set the analysis time-step
    sf = units.g                                                             # Set the scaling factor (if records are in g, a scaling factor of 9.81 m/s2 must be used to be consistent with opensees) 
    collLimit = 10.00                                                        # Set a large number for numerical collapse
    control_nodes, coll_index, peak_drift, peak_accel, max_peak_drift, max_peak_drift_dir, max_peak_drift_loc, max_peak_accel, max_peak_accel_dir, max_peak_accel_loc, peak_disp = model.do_nrha_analysis(fnames, dt_gm, sf, t_max, dt_ansys, collLimit, outDir)

    #######################            
    ### Store the analysis 
    #######################
    sdof_coll_index_list.append(coll_index)
    sdof_peak_drift_list.append(peak_drift)
    sdof_peak_accel_list.append(peak_accel)
    sdof_peak_disp_list.append(peak_disp)
    sdof_max_peak_drift_list.append(max_peak_drift)
    sdof_max_peak_drift_dir_list.append(max_peak_drift_dir)
    sdof_max_peak_drift_loc_list.append(max_peak_drift_loc)
    sdof_max_peak_accel_list.append(max_peak_accel)
    sdof_max_peak_accel_dir_list.append(max_peak_accel_dir)
    sdof_max_peak_accel_loc_list.append(max_peak_accel_loc)
    #######################

    print('================================================================')
    print('============== ANALYSIS COMPLETED: {:d} out {:d} =================='.format(i+1, len(gmrs)))
    print('================================================================')

# 5. Postprocess the Cloud Analysis

In [ ]:
### Process the cloud analysis results for PGA and SA(T1)
sdof_imvector1, sdof_edpvector1 = cloudAnalysis(pga, sdof_max_peak_drift_list)
mdof_imvector1, mdof_edpvector1 = cloudAnalysis(pga, mdof_max_peak_drift_list)

sdof_imvector2, sdof_edpvector2 = cloudAnalysis(sa, sdof_max_peak_drift_list)
mdof_imvector2, mdof_edpvector2 = cloudAnalysis(sa, mdof_max_peak_drift_list)


### Plot the results for Peak Ground Accelerartion
# Plot the sdof cloud and regression
plt.scatter(sdof_max_peak_drift_list, pga, alpha = 0.5)
plt.plot(sdof_edpvector1, sdof_imvector1, linewidth=8)
# Plot the mdof cloud and regression
plt.scatter(mdof_max_peak_drift_list, pga, alpha = 0.5)
plt.plot(mdof_edpvector1, mdof_imvector1, linewidth=8)

plt.xlabel(r'Maximum Peak Storey Drift, $\theta_{max}$ [%]')
plt.ylabel(r'Peak Ground Acceleration, PGA [g]')
plt.grid(visible=True, which='major')
plt.grid(visible=True, which='minor')
plt.xscale('log')
plt.yscale('log')
plt.ylim([5e-3, 1e3])
plt.show()

### Plot the results for Spectral Acceleration at 0.3s
# Plot the sdof cloud and regression
plt.scatter(sdof_max_peak_drift_list, sa, alpha = 0.5)
plt.plot(sdof_edpvector2, sdof_imvector2, linewidth=8)
# Plot the mdof cloud and regression
plt.scatter(mdof_max_peak_drift_list, sa, alpha = 0.5)
plt.plot(mdof_edpvector2, mdof_imvector2, linewidth=8)

plt.xlabel(r'Maximum Peak Storey Drift, $\theta_{max}$ [%]')
plt.ylabel(r'Spectral Acceleration, Sa(T=0.3s) [g]')
plt.grid(visible=True, which='major')
plt.grid(visible=True, which='minor')
plt.xscale('log')
plt.yscale('log')
plt.ylim([5e-3, 1e3])
plt.show()